In [1]:
import os
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)

/home/bruno/Desktop/coarse-discourse-validation


In [44]:
import pandas as pd
import nltk
import numpy as np

from collections import Counter

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/bruno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df_path = os.path.join(PROJ_ROOT, 'data', 'interim', 'clean_data.pkl')

df = pd.read_pickle(df_path)

df.head()

,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...


In [3]:
df_sample = df.iloc[0:1000]

df_sample.head()

,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...


# Features Exploration

In here, we'll be working on the features described on the paper.

## Structure

> We calculate several features related to the structure of the comment and its position. One feature is the depth of the comment according to Reddit’s threaded structure, which we collect as both a raw count and normalized by the number of comments in the discussion. We also calculate number of sentences, number of words, and number of characters of both the body and the title of the comment. We computed these values for both the current comment and the parent comment.

### Post depth

In [4]:
df_sample['post_depth'].value_counts()

1.0     412
2.0     210
3.0     129
0.0     107
4.0      73
5.0      31
6.0      15
7.0       8
8.0       6
10.0      5
9.0       4
Name: post_depth, dtype: int64

In [5]:
# df_sample['post_depth_normalized'] = df_sample['post_depth'] / df_sample['comments_in_discussion']

### Number of sentences

We use NLTK.tokenize to get setences tokenized and then counted.

In [6]:
df_sample['body'].iloc[42]

"You won't regret keeping it. As far as longevity goes, it is almost indefinite. You will wear out parts like springs, mags, and maybe even the grips. The slide, frame, and barrel will likely outlive you. Colt makes great guns. How many companies who produced pistols 100 years ago still have their products found on gun shop shelves in working and safe condition?"

In [7]:
nltk.download('punkt')

nltk.tokenize.sent_tokenize(df_sample['body'].iloc[42])

[nltk_data] Downloading package punkt to /home/bruno/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


["You won't regret keeping it.",
 'As far as longevity goes, it is almost indefinite.',
 'You will wear out parts like springs, mags, and maybe even the grips.',
 'The slide, frame, and barrel will likely outlive you.',
 'Colt makes great guns.',
 'How many companies who produced pistols 100 years ago still have their products found on gun shop shelves in working and safe condition?']

In [8]:
sent_number = lambda b : len(nltk.tokenize.sent_tokenize(b))

df_sample['n_sentences'] = df_sample['body'].map(sent_number)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5


### Number of words

In [9]:
nltk.tokenize.word_tokenize(df_sample['body'].iloc[42])

['You',
 'wo',
 "n't",
 'regret',
 'keeping',
 'it',
 '.',
 'As',
 'far',
 'as',
 'longevity',
 'goes',
 ',',
 'it',
 'is',
 'almost',
 'indefinite',
 '.',
 'You',
 'will',
 'wear',
 'out',
 'parts',
 'like',
 'springs',
 ',',
 'mags',
 ',',
 'and',
 'maybe',
 'even',
 'the',
 'grips',
 '.',
 'The',
 'slide',
 ',',
 'frame',
 ',',
 'and',
 'barrel',
 'will',
 'likely',
 'outlive',
 'you',
 '.',
 'Colt',
 'makes',
 'great',
 'guns',
 '.',
 'How',
 'many',
 'companies',
 'who',
 'produced',
 'pistols',
 '100',
 'years',
 'ago',
 'still',
 'have',
 'their',
 'products',
 'found',
 'on',
 'gun',
 'shop',
 'shelves',
 'in',
 'working',
 'and',
 'safe',
 'condition',
 '?']

In [10]:
words_number = lambda b : len(nltk.tokenize.word_tokenize(b))

df_sample['n_words'] = df_sample['body'].map(words_number)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences,n_words
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29,499
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,22
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4,82
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,26
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5,115


### Number of characters

Here we create a tokenizer from a RegEx to remove punctuaions and whitespaces, so we can count only characters.

In [11]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

words = tokenizer.tokenize(df_sample['body'].iloc[42])

sum(map(len, words))

289

In [12]:
char_number = lambda b : sum(map(len, tokenizer.tokenize(b)))

df_sample['n_char'] = df_sample['body'].map(char_number)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences,n_words,n_char
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29,499,2009
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,22,74
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4,82,339
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,26,84
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5,115,434


### Parents number of sentences, words and characters

In [13]:
df_sample['parent_n_sentences'] = 0
df_sample['parent_n_words'] = 0
df_sample['parent_n_char'] = 0

for i, row in df_sample.iterrows():
    try:
        parent = df_sample[(df_sample['thread_title'] == row['thread_title'])][df_sample['id'] == row['in_reply_to']].iloc[0]
        df_sample.at[i, 'parent_n_sentences'] = parent['n_sentences']
        df_sample.at[i, 'parent_n_words'] = parent['n_words']
        df_sample.at[i, 'parent_n_char'] = parent['n_char']
    except:
        pass
    
df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences,n_words,n_char,parent_n_sentences,parent_n_words,parent_n_char
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29,499,2009,0,0,0
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,22,74,29,499,2009
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4,82,339,2,22,74
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,26,84,4,82,339
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5,115,434,0,0,0


## Author

> We collect features about the author of the comment, including a binary feature for whether the current commenter is also the commenter of the initial post and a binary feature for whether the current commenter is the same as the parent commenter.

In [14]:
def is_thread_author(row):
    if row['author'] == '[deleted]' :
        return False
    else:
        return row['author'] == row['thread_author']

df_sample['is_thread_author'] = df_sample.apply(is_thread_author, axis=1)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences,n_words,n_char,parent_n_sentences,parent_n_words,parent_n_char,is_thread_author
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29,499,2009,0,0,0,True
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,22,74,29,499,2009,False
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4,82,339,2,22,74,True
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,26,84,4,82,339,False
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5,115,434,0,0,0,True


In [15]:
def is_parent_author(row):
    try:
        parent_author = df_sample[df_sample['id']==row['in_reply_to']].iloc[0]['author']
        
        return row['author'] == parent_author and row['author'] != '[deleted]'
    except IndexError:
        return False

df_sample['is_parent_author'] = df_sample.apply(is_parent_author, axis=1)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,...,thread_title,url,n_sentences,n_words,n_char,parent_n_sentences,parent_n_words,parent_n_char,is_thread_author,is_parent_author
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,...,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29,499,2009,0,0,0,True,False
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,...,DTX120: #87 - Nashville,NaN,2,22,74,29,499,2009,False,False
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,...,DTX120: #87 - Nashville,NaN,4,82,339,2,22,74,True,False
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,...,DTX120: #87 - Nashville,NaN,2,26,84,4,82,339,False,False
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,...,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5,115,434,0,0,0,True,False


In [28]:
df_sample['is_parent_author'].value_counts()

False    982
True      18
Name: is_parent_author, dtype: int64

In [29]:
df_sample['is_thread_author'].value_counts()

False    756
True     244
Name: is_thread_author, dtype: int64

## Thread

> We calculate features that are the same across all comments in the thread. One feature is the total number of comments in the discussion. Another is the number of unique branches in the discussion tree. We also record whether the discussion originated as a self-post or a link-post. Finally, we collect the average length of all the branches or threads of discussion in the discussion tree.

Most of these may be included in the pre-processing step.

## Community

>We have a feature naming the subreddit that the thread came from, as some subreddits have a greater porportion of some types of discourse and not others.

In [40]:
df_sample['subreddit'] = df_sample['subreddit'].astype('category')

df_sample['subreddit']

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0        100movies365days
1        100movies365days
2        100movies365days
3        100movies365days
4                 100sets
5                 100sets
7                 100sets
8                 100sets
9                 100sets
10                100sets
11           1200isplenty
12           1200isplenty
13           1200isplenty
14           1200isplenty
15           1200isplenty
16           1200isplenty
17           1200isplenty
18           1200isplenty
19                   1911
20                   1911
21                   1911
22                   1911
23                   1911
24                   1911
25                   1911
26                   1911
27                   1911
28                   1911
29                   1911
30                   1911
              ...        
980         adviceanimals
981         adviceanimals
982         adviceanimals
983         adviceanimals
984         adviceanimals
985         adviceanimals
986         adviceanimals
987         

## Content + Punctuation

>We collect unigrams, bigrams, and trigrams from the text of the comment. If the comment has a title, in the case of the initial post, then the n-grams of the title are counted separately from the n-grams of the body. We use a word tokenizer that tokenizes punctuation instead of stripping it so that we count potentially important punctuation like question marks or exclamation points. We use TF-IDF weighting and set a minimum document frequency of 50 comments.

In [56]:
unigrams_frequency = Counter()
bigrams_frequency = Counter()
trigrams_frequency = Counter()

for body in df_sample['body']:
    tokens = nltk.word_tokenize(body)
    unigrams = tokens
    bigrams = nltk.bigrams(tokens)
    trigrams = nltk.trigrams(tokens)
    
    unigrams_frequency += Counter(unigrams)
    bigrams_frequency += Counter(bigrams)
    trigrams_frequency += Counter(trigrams)

In [57]:
unigrams_frequency.most_common(10)

[('.', 2043),
 (',', 1346),
 ('I', 1326),
 ('the', 1219),
 ('to', 1097),
 ('a', 887),
 ('and', 794),
 ('you', 712),
 ('it', 586),
 ('of', 495)]

In [58]:
bigrams_frequency.most_common(10)

[(('.', 'I'), 291),
 (('I', "'m"), 147),
 ((',', 'and'), 146),
 ((',', 'but'), 136),
 ((',', 'I'), 129),
 (('!', '!'), 118),
 (('do', "n't"), 102),
 (('http', ':'), 99),
 ((']', '('), 94),
 (('of', 'the'), 83)]

In [59]:
trigrams_frequency.most_common(10)

[(('!', '!', '!'), 68),
 (('(', 'http', ':'), 64),
 ((']', '(', 'http'), 61),
 (('I', 'do', "n't"), 44),
 (('.', 'I', "'m"), 41),
 (('.', 'If', 'you'), 29),
 ((',', 'but', 'I'), 27),
 (('.', 'It', "'s"), 26),
 ((']', '(', 'https'), 25),
 (('(', 'https', ':'), 25)]